# Installing required packages

In [1]:
!pip install tensorflow

You should consider upgrading via the 'F:\mentalhealthCanada\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
!pip install nltk

You should consider upgrading via the 'F:\mentalhealthCanada\venv\Scripts\python.exe -m pip install --upgrade pip' command.


# Importing required libraries

In [3]:
import json
import pickle
import random
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

f:\mentalhealthcanada\venv\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
f:\mentalhealthcanada\venv\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
f:\mentalhealthcanada\venv\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Making lists to store words, classes and documents

In [4]:
vocabulary_list = []
categories_list = []
document_list = []

# List of characters to ignore

In [5]:
characters = ['?', '!']

# Opening and Loading the Dataset 

In [6]:
dataset = open('mentalhealthCanada.json').read()
intents = json.loads(dataset)

# Updating the vocabulary, category and documents lists

In [7]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #Tokenizing each word from pattern into the vocabulary list
        w = nltk.word_tokenize(pattern)
        vocabulary_list.extend(w)
        
        #Adding documents into the collection (corpus)
        document_list.append((w, intent['tag']))

        #Adding unique tags into the categories list
        if intent['tag'] not in categories_list:
            categories_list.append(intent['tag'])

# Text pre-processing (lower case, remove non-alphanumeric characters and lemmatize text)

In [8]:
lemma = WordNetLemmatizer()
vocabulary = [lemma.lemmatize(w.lower()) for w in vocabulary_list if w not in characters]
vocabulary = sorted(list(set(vocabulary)))

# Sorting the categories list

In [9]:
categories = sorted(list(set(categories_list)))

# Documents list is a combination between intents and patterns

In [10]:
print (len(document_list), "documents")

177 documents


# Categories list is the intents

In [11]:
print (len(categories), "classes", categories)

57 classes ['alberta_support', 'anxiety_causes', 'anxiety_disorders', 'anxiety_symptoms', 'anxiety_treatment', 'bc_support', 'bipolar_causes', 'bipolar_disorder', 'bipolar_symptoms', 'bipolar_treatment', 'canada_helpline', 'canada_support', 'depression', 'depression_causes', 'depression_symptoms', 'depression_treatment', 'eating_causes', 'eating_disorder', 'eating_symptoms', 'eating_treatment', 'generalized_disorder', 'goodbye', 'greeting', 'helpline_select', 'helpline_select_app', 'helpline_select_group', 'manitoba_support', 'mental_health', 'mental_health_illness', 'mental_illness', 'mental_illness_causes', 'mental_illness_symptoms', 'mental_illness_types', 'mental_treatment', 'nb_support', 'noanswer', 'ns_support', 'obsessive_disorder', 'ontario_support', 'options', 'panic_disorder', 'personality_causes', 'personality_disorders', 'personality_symptoms', 'personality_treatment', 'phobias', 'post_disorder', 'post_symptoms', 'quebec_support', 'saskatchewan_support', 'schizophrenia', 's

# Vocabulary list is all the unique words

In [12]:
print (len(vocabulary), "Unique lemmatized words", vocabulary)

126 Unique lemmatized words ["'s", ',', '.', '1', '2', '3', '4', 'a', 'about', 'alberta', 'and', 'anxiety', 'any', 'anyone', 'apps', 'are', 'available', 'awesome', 'better', 'bipolar', 'both', 'british', 'brunswick', 'bye', 'call', 'can', 'canada', 'care', 'cause', 'chat', 'chatting', 'columbia', 'concern', 'contact', 'could', 'crisis', 'day', 'depression', 'detail', 'different', 'disorder', 'do', 'eating', 'factor', 'for', 'generalized', 'get', 'give', 'good', 'goodbye', 'group', 'have', 'health', 'hello', 'help', 'helpful', 'helping', 'helpline', 'hey', 'hi', 'how', 'i', 'illness', 'in', 'information', 'is', 'issue', 'later', 'manitoba', 'me', 'mental', 'more', 'my', 'new', 'next', 'nice', 'nova', 'number', 'obsessive-compulsive', 'of', 'on', 'online', 'ontario', 'panic', 'peer', 'personality', 'phobia', 'phone', 'post-traumatic', 'provide', 'province', 'province-wise', 'quebec', 'regarding', 'resource', 'same', 'saskatchewan', 'schizophrenia', 'scotia', 'see', 'service', 'should', '

# Converting the vocabulary and category lists from objects into byte-stream and storing in respective pickle files

In [13]:
pickle.dump(vocabulary,open('vocabulary.pkl','wb'))
pickle.dump(categories,open('categories.pkl','wb'))

# Creating a list for the training data

In [14]:
traininglist = []

# Creating and empty array to store the outputs

In [15]:
output_empty = [0] * len(categories)

# Training set will consist of bag of words for each text sentence

In [16]:
for doc in document_list:
    # Creating the bag of words list
    bagofwords = []
    
    # Creating a list of tokenized words for the pattern
    pattern_words = doc[0]
    
    # Lemmatizing each word - Creating base lemma words in an attempt to represent relative words
    pattern_words = [lemma.lemmatize(word.lower()) for word in pattern_words]
    
    # Creating the bag of words array with 1, if word match is found in the current pattern
    for w in vocabulary:
        bagofwords.append(1) if w in pattern_words else bagofwords.append(0)
    
    # Output will be '0' for each tag and '1' will be for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[categories.index(doc[1])] = 1
    traininglist.append([bagofwords, output_row])

# Shuffling the features and turning them into a numpy array

In [17]:
random.shuffle(traininglist)
training = np.array(traininglist)

C:\Users\Admin\AppData\Local\Temp/ipykernel_5004/3339943570.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training = np.array(traininglist)


# Creating the training and testing lists. X is the patterns, Y is the intents

In [18]:
X_train = list(training[:,0])
Y_train = list(training[:,1])
print("Training data created")

Training data created


# Creating the model - 3 layers. First layer contains 128 neurons, second layer contains 64 neurons and third output layer contains number of neurons that are equal to number of intents for predicting the output intent with softmax

In [19]:
# Sequential model code adapted from 'Keras' documentation
# https://keras.io/guides/sequential_model/ [Accessed 16 July 2021].

botmodel = Sequential()
botmodel.add(Dense(128, input_shape=(len(X_train[0]),), activation='relu'))
botmodel.add(Dropout(0.5))
botmodel.add(Dense(64, activation='relu'))
botmodel.add(Dropout(0.5))
botmodel.add(Dense(len(Y_train[0]), activation='softmax'))

# Compiling the model. Using stochastic gradient descent with Nesterov accelerated gradient gives good results for the model

In [20]:
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
botmodel.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# Fitting and saving the model for future use

In [21]:
createbot = botmodel.fit(np.array(X_train), np.array(Y_train), epochs=300, batch_size=6, verbose=1)
botmodel.save('mentalhealthbot_model.h5', createbot)
print("model created")

Epoch 1/300
30/30 [==============================] - 2s 4ms/step - loss: 4.0738 - accuracy: 0.0169
Epoch 2/300
30/30 [==============================] - 0s 5ms/step - loss: 4.0261 - accuracy: 0.0395
Epoch 3/300
30/30 [==============================] - 0s 5ms/step - loss: 3.9352 - accuracy: 0.0452
Epoch 4/300
30/30 [==============================] - 0s 5ms/step - loss: 3.8691 - accuracy: 0.0678
Epoch 5/300
30/30 [==============================] - 0s 4ms/step - loss: 3.8066 - accuracy: 0.0621
Epoch 6/300
30/30 [==============================] - 0s 6ms/step - loss: 3.6923 - accuracy: 0.0960 ETA: 0s - loss: 3.6769 - accuracy: 0.
Epoch 7/300
30/30 [==============================] - 0s 7ms/step - loss: 3.5459 - accuracy: 0.1186
Epoch 8/300
30/30 [==============================] - 0s 5ms/step - loss: 3.4423 - accuracy: 0.1186
Epoch 9/300
30/30 [==============================] - 0s 5ms/step - loss: 3.2949 - accuracy: 0.1638
Epoch 10/300
30/30 [==============================] - 0s 5ms/step - los

30/30 [==============================] - 0s 4ms/step - loss: 0.5598 - accuracy: 0.8079
Epoch 79/300
30/30 [==============================] - 0s 3ms/step - loss: 0.5828 - accuracy: 0.8305
Epoch 80/300
30/30 [==============================] - 0s 4ms/step - loss: 0.5812 - accuracy: 0.8249
Epoch 81/300
30/30 [==============================] - 0s 4ms/step - loss: 0.4869 - accuracy: 0.8362
Epoch 82/300
30/30 [==============================] - 0s 4ms/step - loss: 0.4981 - accuracy: 0.8418
Epoch 83/300
30/30 [==============================] - 0s 4ms/step - loss: 0.5082 - accuracy: 0.8418
Epoch 84/300
30/30 [==============================] - 0s 4ms/step - loss: 0.4267 - accuracy: 0.8927
Epoch 85/300
30/30 [==============================] - 0s 4ms/step - loss: 0.5149 - accuracy: 0.8362
Epoch 86/300
30/30 [==============================] - 0s 5ms/step - loss: 0.5116 - accuracy: 0.8305
Epoch 87/300
30/30 [==============================] - 0s 3ms/step - loss: 0.4191 - accuracy: 0.8418
Epoch 88/300


30/30 [==============================] - 0s 6ms/step - loss: 0.2867 - accuracy: 0.9209
Epoch 158/300
30/30 [==============================] - 0s 3ms/step - loss: 0.3469 - accuracy: 0.8418
Epoch 159/300
30/30 [==============================] - 0s 4ms/step - loss: 0.2429 - accuracy: 0.9322
Epoch 160/300
30/30 [==============================] - 0s 4ms/step - loss: 0.2181 - accuracy: 0.9492
Epoch 161/300
30/30 [==============================] - 0s 3ms/step - loss: 0.2496 - accuracy: 0.9096
Epoch 162/300
30/30 [==============================] - 0s 5ms/step - loss: 0.2387 - accuracy: 0.9209
Epoch 163/300
30/30 [==============================] - 0s 5ms/step - loss: 0.2158 - accuracy: 0.9209
Epoch 164/300
30/30 [==============================] - 0s 4ms/step - loss: 0.2510 - accuracy: 0.9209
Epoch 165/300
30/30 [==============================] - 0s 4ms/step - loss: 0.2837 - accuracy: 0.8983
Epoch 166/300
30/30 [==============================] - 0s 5ms/step - loss: 0.2608 - accuracy: 0.9153
Epoc

30/30 [==============================] - 0s 5ms/step - loss: 0.1933 - accuracy: 0.9266
Epoch 239/300
30/30 [==============================] - 0s 4ms/step - loss: 0.2813 - accuracy: 0.8983
Epoch 240/300
30/30 [==============================] - 0s 4ms/step - loss: 0.1883 - accuracy: 0.9435
Epoch 241/300
30/30 [==============================] - 0s 4ms/step - loss: 0.1831 - accuracy: 0.9153
Epoch 242/300
30/30 [==============================] - 0s 4ms/step - loss: 0.1906 - accuracy: 0.9492
Epoch 243/300
30/30 [==============================] - 0s 4ms/step - loss: 0.3090 - accuracy: 0.9096
Epoch 244/300
30/30 [==============================] - 0s 4ms/step - loss: 0.2621 - accuracy: 0.9209
Epoch 245/300
30/30 [==============================] - 0s 6ms/step - loss: 0.1831 - accuracy: 0.9322
Epoch 246/300
30/30 [==============================] - 0s 4ms/step - loss: 0.1654 - accuracy: 0.9492
Epoch 247/300
30/30 [==============================] - 0s 4ms/step - loss: 0.2508 - accuracy: 0.9266
Epoc

# Summary of the model

In [22]:
botmodel.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               16256     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 57)                3705      
Total params: 28,217
Trainable params: 28,217
Non-trainable params: 0
_________________________________________________________________
